#  Preparation of T-cell Enriched annData Object

- Cellranger multi (10x Genomics, v7.0.0) and Cellranger aggregate (10x Genomics, v7.0.0) were used on GEX and VDJ libraries from 5 lung samples and 6 liver samples. 
- The Cellranger multi pipeline was run on the Terra Cloud Computing Platform can be found here: https://portal.firecloud.org/?return=terra#methods/Alignment/Cellranger_Multi/11.
- The Cellranger aggregate pipeline was run on the Terra Cloud Computing Platformcan be found here: https://portal.firecloud.org/?return=terra#methods/Alignment/Cellranger_Aggregate_For_Multi_Samples/16
- During QC, one liver sample was thrown out due to poor quality (gem well 5). Thus, 5 lung samples and 5 liver samples were included in the analysis
- To remove ambient RNA, we then ran the "remove-background" command in Cellbender. Default settings were used, with fpr = "0.01" and total_droplets_included = 40000. The pipeline was run on the Terra Cloud Computing platform and can be found here: https://portal.firecloud.org/?return=terra#methods/cellbender/remove-background/11
- The resulting data was incorporated into a annData object (v0.9.2)
- During manual QC, only Cell-IDs with mitochondrial DNA < 10%, a minimum of 750 genes, and at least 2000 transcripts but no greater than 25000 transcripts were retained
- Using pybiomart (version 0.2.0), we extracted Ensembl Biomart was used to identify Ensembl gene IDs, gene names, gene biotypes, and chromosome names. Pybiomart dataset settings were as follows: name = 'mmulatta_gene_ensembl, host='http://www.ensembl.org'
- Only genes that were "protein_coding" and "miRNA" gene types were retained. Genes from the Y chromosome were removed.
- Scanpy (insert version) was used to identify highly variable genes from raw counts, with "n_top_genes = 5000", "flavor = seurat_v3", and "subset=False" 
- Only highly variable genes as well as the following T-cell markers were carried forward: "CD3E", "TRAC", "CD4", and "CD8A".
- The .hdf5 file containing this filtered object has been provided as **"01_pre_filtered_object.hdf5"**. It has been modified for usability.
- A VAE model was trained using the pre_filtered_object via scVI. 
- There were two experimental batches. Categorical covariate keys were experimental batch as well as animal.
- Non T-cell enriched clusters were thrown out and a new VAE model was trained, followed by another round of scVI clustering. The final filtered object is provided as **"02_final_filtered_object.hdf5"**. It has been modified for usability. It is this object that was used for all downstream analysis.
- The two VAE models are provided as **"scVI_Model_01"** and **"scVI_Model_02"**. Please note that the models were initially trained on <v0.15.0. To use the models in a newer version, the "convert_legacy_save" function must be run to convert to an updated format.

In [ ]:
import numpy as np #v1.24.4
import pandas as pd #v2.1.1
import anndata #v0.9.2
import scanpy as sc #v1.9.6
import scvi #v<0.15.0
import random

c_iSeed = 6161904
np.random.seed(c_iSeed)
random.seed(c_iSeed)
scvi.settings.num_threads = 8
scvi.settings.progress_bar_style = "rich"
scvi.settings.seed = c_iSeed

In [ ]:
pre_filtered_object = anndata.read_h5ad(filename=___) #Replace ___ with path to file "01_pre_filtered_object.hdf5"

In [ ]:
# Set scVI Model Parameters
scvi.model.SCVI.setup_anndata(pre_filtered_object,
    layer="raw_counts",
    categorical_covariate_keys=["experimental_batch", "animal_id"]
)

scVI_Model_01 = scvi.model.SCVI(pre_filtered_object,
                          n_layers=2,
                          n_latent=30,
                          gene_likelihood="nb")

n_val_max = 40000
if 0.1 * pre_filtered_object.n_obs < n_val_max:
    train_size = 0.9
else:
    train_size = 1 - (n_val_max/pre_filtered_object.n_obs)

# Train VAE
scVI_Model_01.train(train_size=train_size,
              check_val_every_n_epoch=10,
              early_stopping=True,    
              early_stopping_patience=45,
              max_epochs=10000, 
              batch_size=1024,    
              limit_train_batches=20)

# Generate scVI Clusters
def Prepare_scVI_Latent_Space(annIn,vaeIn):

    # Get Latent Space
    annIn.obsm["scVI_Latent_Space"] = vaeIn.get_latent_representation()
            
    # Cluster
    sc.pp.neighbors(annIn, use_rep="scVI_Latent_Space")
    sc.tl.umap(annIn) #For visualization downstream
    sc.tl.leiden(annIn, key_added="scVI_clusters")
    
    return

Prepare_scVI_Latent_Space(pre_filtered_object,scVI_Model_01)

# Get Normalized Expression
pre_filtered_object.layers["scvi_norm_counts"] = scVI_Model_01.get_normalized_expression(library_size=10e4)

In [ ]:
# Throw Out Non-T Cell Enriched Clusters 
pre_filtered_object = pre_filtered_object[pre_filtered_object.obs["scVI_clusters"] != 16]
pre_filtered_object = pre_filtered_object[pre_filtered_object.obs.query("scVI_clusters != '16'").index]
pre_filtered_object = pre_filtered_object[[r != "16" for r in pre_filtered_object.obs.scVI_clusters]]

pre_filtered_object = pre_filtered_object[pre_filtered_object.obs["scVI_clusters"] != 17]
pre_filtered_object = pre_filtered_object[pre_filtered_object.obs.query("scVI_clusters != '17'").index]
pre_filtered_object = pre_filtered_object[[r != "17" for r in pre_filtered_object.obs.scVI_clusters]]

In [ ]:
# Set scVI Model Parameters
scvi.model.SCVI.setup_anndata(pre_filtered_object,
    layer="raw_counts",
    categorical_covariate_keys=["experimental_batch", "animal_id"]
)

scVI_Model_02 = scvi.model.SCVI(pre_filtered_object,
                          n_layers=2,
                          n_latent=30,
                          gene_likelihood="nb")

n_val_max = 40000
if 0.1 * post_scVI_object.n_obs < n_val_max:
    train_size = 0.9
else:
    train_size = 1 - (n_val_max/pre_filtered_object.n_obs)

# Train VAE
scVI_Model_02.train(train_size=train_size,
              check_val_every_n_epoch=10,
              early_stopping=True,    
              early_stopping_patience=45,
              max_epochs=10000, 
              batch_size=1024,    
              limit_train_batches=20)

# Generate scVI Clusters
def Prepare_scVI_Latent_Space(annIn,vaeIn):

    # Get Latent Space
    annIn.obsm["scVI_Latent_Space"] = vaeIn.get_latent_representation()
            
    # Cluster
    sc.pp.neighbors(annIn, use_rep="scVI_Latent_Space")
    sc.tl.umap(annIn) #For visualization downstream
    sc.tl.leiden(annIn, key_added="scVI_clusters")
    
    return

Prepare_scVI_Latent_Space(pre_filtered_object,scVI_Model_02)

# Get Normalized Expression
pre_filtered_object.layers["scvi_norm_counts"] = scVI_Model_02.get_normalized_expression(library_size=10e4)

# The final annData object is provided as "02_final_filtered_object.hdf5" and was used for all downstream analysis